In [51]:
import os
import importlib.util
from datetime import datetime
from ast import literal_eval
from transformers import pipeline
import json
import sys
import numpy as np

# Import modules into path
current_dir = os.getcwd()
module_path = os.path.join(current_dir, 'src', 'nlp_models.py')
module_path_g = os.path.join(current_dir, 'src', 'llms.py')

spec = importlib.util.spec_from_file_location("nlp_models", module_path)
spec_g = importlib.util.spec_from_file_location("llms", module_path_g)

# Load the modules
nlp_module = importlib.util.module_from_spec(spec)
sys.modules["nlp_models"] = nlp_module
spec.loader.exec_module(nlp_module)

llms = importlib.util.module_from_spec(spec_g)
sys.modules["llms"] = llms
spec_g.loader.exec_module(llms)


In [6]:
from characterize import Characterizer
from ast import literal_eval
import pandas as pd
import numpy as np
import os

def characterize_text(text):
    char = Characterizer()
    nouns, verbs, adjs = char.extract_word_groups(text)

    nouns_translated = char.translate_mymemory(nouns)
    noun_genders = char.get_gender(nouns)
    
    inf = char.get_infinitive_pylefff(verbs)
    verbs_translated = char.translate_mymemory(inf)
    
    adjs_translated = char.translate_mymemory(adjs)

    print(len(nouns), len(noun_genders))

    noun_df = pd.DataFrame({'word' :nouns, 
                       'translation' : nouns_translated, 
                       'type' : ['noun' for n in range(len(nouns))]
                      })
    
    verb_df = pd.DataFrame({'word' : inf, 
                       'translation' : verbs_translated, 
                       'type' : ['verb' for n in range(len(inf))]
                      })

    
    adj_df = pd.DataFrame({'word' :adjs, 
                       'translation' : adjs_translated, 
                       'type' : ['verb' for n in range(len(adjs))]
                      })
    df = pd.concat([noun_df, verb_df, adj_df], axis = 0)
    
    return df


In [7]:
# Test text - the first paragraph from a Le Monde article on Romanian politics
text = """ La Roumanie a un nouveau gouvernement composé en grande partie des mêmes figures que l’équipe précédente. 
Lundi 23 décembre, le président roumain, Klaus Iohannis, a reconduit le social-démocrate Marcel Ciolacu, 57 ans, à la tête d’un gouvernement de crise. 
Premier ministre depuis juin 2023, M. Ciolacu prend les rênes d’un exécutif remanié après les élections législatives du 1er décembre, organisées dans un contexte de turbulence politique marqué par l’annulation, cinq jours plus tard, de l’élection présidentielle par la Cour constitutionnelle. 
« Le mandat que j’ai reçu aujourd’hui est un honneur. Nous traversons une période complexe et j’espère que nous avons tous appris les leçons du passé », a déclaré M. Ciolacu après l’annonce de sa reconduction."""


from datetime import datetime
st = datetime.now()

df = characterize_text(text)

print(f"Characterization took {(datetime.now() - st)} total")

df

Device set to use cpu


33 155
Characterization took 0:01:24.186310 total


,word,translation,type
0,gouvernement,government,noun
1,figures,FIGURES,noun
2,équipe,Team,noun
3,Lundi,Monday,noun
4,décembre,December,noun
5,président,president,noun
6,social,social,noun
7,démocrate,DEM,noun
8,ans,years,noun
9,tête,head,noun


### Example with the First Chapter of Doestoevsky's Le Joueur (The Gambler)

In [9]:
import re

# Load the text file 
with open("le_joueur.txt", "r") as file:
    text = file.read()

# Splitting on 'I' is problematic for Chapter I
first_chapter = text.split("""Je suis enfin revenu""")[1].split("II")[0]
first_chapter = "Je suis enfin revenu" + first_chapter

# print first sentence
first_chapter.split('.')[0] + '.'

'Je suis enfin revenu de mon absence de deux semaines.'

In [10]:
st = datetime.now()
df = characterize_text(first_chapter)
print(f"Characterization took {(datetime.now() - st)} total")
df.head()

Device set to use cpu


60 235
Characterization took 0:03:56.100804 total


,word,translation,type
0,absence,absence,noun
1,semaines,weeks,noun
2,jours,days,noun
3,impatience,Impatience,noun
4,général,general,noun
